### INTRODUCTION - This is an extension of previous mnist_v2 model for image classification with neural networks. This is a sequential model with one input layer, two hidden layers having consecutive dropout layers and an output layer which further introduces the concept of combatting overfitting using dropout layers(works by randomly setting the outgoing edges of hidden units (neurons that make up hidden layers) to 0 at each update of the training phase). It also differs in epochs which is increased back to 200 from 50.

In [ ]:
import tensorflow as tf
import numpy as np
from tensorflow import keras

In [ ]:
# network and training
EPOCHS = 200
BATCH_SIZE = 128
VERBOSE = 1
NB_CLASSES = 10   # number of outputs = number of digits
N_HIDDEN = 128
VALIDATION_SPLIT=0.2 # how much TRAIN is reserved for VALIDATION
DROPOUT = 0.3   # Percent of neurons to be ignored in dropout

In [ ]:
# loading MNIST dataset
# verify
# the split between train and test is 60,000, and 10,000 respectly 
# one-hot is automatically applied
mnist = keras.datasets.mnist
(X_train, Y_train), (X_test, Y_test) = mnist.load_data()

11493376/11490434 [==============================] - 0s 0us/step


In [ ]:
#X_train is 60000 rows of 28x28 values --> reshaped in 60000 x 784
RESHAPED = 784

X_train = X_train.reshape(60000, RESHAPED)
X_test = X_test.reshape(10000, RESHAPED)
X_train = X_train.astype('float32')
X_test = X_test.astype('float32')


In [ ]:
#normalize in [0,1]
X_train, X_test = X_train / 255.0, X_test / 255.0
print(X_train.shape[0], 'train samples')
print(X_test.shape[0], 'test samples')

60000 train samples
10000 test samples


In [ ]:
#one-hot
Y_train = tf.keras.utils.to_categorical(Y_train, NB_CLASSES)
Y_test = tf.keras.utils.to_categorical(Y_test, NB_CLASSES)

In [ ]:
#build the model
model = tf.keras.models.Sequential()
model.add(keras.layers.Dense(N_HIDDEN,
    input_shape=(RESHAPED,),
    name='dense_layer', activation='relu'))

'''A simple and powerful regularization technique for neural networks and deep learning models is dropout. 
Dropout is a technique where randomly selected neurons are ignored during training. They are “dropped-out” randomly. 
This means that their contribution to the activation of downstream neurons is temporally removed on the forward pass 
and any weight updates are not applied to the neuron on the backward pass.
Dropout is easily implemented by randomly selecting nodes to be dropped-out with a given probability (e.g. 20%) each 
weight update cycle. This is how Dropout is implemented in Keras. Dropout is only used during the training of a model 
and is not used when evaluating the skill of the model.
For example, if the dropout rate is set to 20%, meaning one in 5 inputs will be randomly excluded from each update cycle.
Reference - https://machinelearningmastery.com/dropout-regularization-deep-learning-models-keras/
'''
model.add(keras.layers.Dropout(DROPOUT))
model.add(keras.layers.Dense(N_HIDDEN,
    name='dense_layer_2', activation='relu'))
model.add(keras.layers.Dropout(DROPOUT))
model.add(keras.layers.Dense(NB_CLASSES,
    name='dense_layer_3', activation='softmax'))

In [ ]:
# summary of the model
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_layer (Dense)          (None, 128)               100480    
_________________________________________________________________
dropout (Dropout)            (None, 128)               0         
_________________________________________________________________
dense_layer_2 (Dense)        (None, 128)               16512     
_________________________________________________________________
dropout_1 (Dropout)          (None, 128)               0         
_________________________________________________________________
dense_layer_3 (Dense)        (None, 10)                1290      
Total params: 118,282
Trainable params: 118,282
Non-trainable params: 0
_________________________________________________________________


In [ ]:
# compiling the model
model.compile(optimizer='SGD', 
              loss='categorical_crossentropy',
              metrics=['accuracy'])

In [ ]:
#training the moodel
model.fit(X_train, Y_train,
        batch_size=BATCH_SIZE, epochs=EPOCHS,
        verbose=VERBOSE, validation_split=VALIDATION_SPLIT)


Epoch 1/200
375/375 [==============================] - 2s 4ms/step - loss: 1.7962 - accuracy: 0.4209 - val_loss: 1.0411 - val_accuracy: 0.7808
Epoch 2/200
375/375 [==============================] - 1s 4ms/step - loss: 0.9901 - accuracy: 0.6985 - val_loss: 0.5681 - val_accuracy: 0.8569
Epoch 3/200
375/375 [==============================] - 1s 4ms/step - loss: 0.7239 - accuracy: 0.7761 - val_loss: 0.4422 - val_accuracy: 0.8832
Epoch 4/200
375/375 [==============================] - 1s 4ms/step - loss: 0.6139 - accuracy: 0.8123 - val_loss: 0.3829 - val_accuracy: 0.8965
Epoch 5/200
375/375 [==============================] - 1s 4ms/step - loss: 0.5414 - accuracy: 0.8365 - val_loss: 0.3466 - val_accuracy: 0.9042
Epoch 6/200
375/375 [==============================] - 1s 4ms/step - loss: 0.4949 - accuracy: 0.8515 - val_loss: 0.3205 - val_accuracy: 0.9097
Epoch 7/200
375/375 [==============================] - 1s 4ms/step - loss: 0.4660 - accuracy: 0.8600 - val_loss: 0.3028 - val_accuracy: 0.9147

In [ ]:
#evalute the model
test_loss, test_acc = model.evaluate(X_test, Y_test)
print('Test accuracy:', test_acc)

313/313 [==============================] - 1s 3ms/step - loss: 0.0753 - accuracy: 0.9764
Test accuracy: 0.9764000177383423


In [ ]:
# making prediction
predictions = model.predict(X_test)

**Observations**

We have achieved slightly improved accuracy with dropout even and here the epochs have been increasaed to 200. training accuracy 97.69,validation accuracy-97.63 and test accuracy - 97.64

It has been frequently observed that networks with random dropout in internal hidden layers can "generalize" better on unseen examples contained in test sets. Intuitively, we can consider this phenomenon as each neuron becoming more capable because it knows it cannot depend on its neighbors. Also, because it forces information to be stored in a redundant way. During testing there is no dropout, so we are now using all our highly tuned neurons. In short, it is generally a good approach to test how a net performs when a dropout function is adopted.

Besides that, note that training accuracy should still be above test accuracy, otherwise, we might be not training for long enough. This is the case in our example and therefore we should increase the number of epochs. But before that we will apply some more optimization techniques.